### ДЗ1  
### Выполнил: _Сидоров Иван_  
### Задание:  
* Доработайте функцию generate, чтобы она работала на триграммах, а не биграммах. То есть подбирала следующее слово текста на основе двух предыдущих, а не одного.
* Напишите функцию, которая подбирает рифмы к слову на основе cmudict

In [ ]:
%pip install nltk numpy

In [1]:
import nltk
import random
import urllib


nltk.download('punkt')
url = 'https://www.gutenberg.org/cache/epub/72839/pg72839.txt'
request = urllib.request.urlopen(url)
text = request.read().decode('utf-8')
text = nltk.Text(nltk.tokenize.word_tokenize(text))

[nltk_data] Downloading package punkt to /Users/ivsidorov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Задание 1

In [33]:
cfd = nltk.ConditionalFreqDist(nltk.bigrams(text))

In [101]:
def generate(first_word: str, cfd: nltk.ConditionalFreqDist, word_count: int = 10) -> str:
    words = [first_word]
    current_word = first_word
    for _ in range(word_count - 1):
        probs = cfd[current_word]
        if probs.keys():
            next_word = random.choices(list(probs.keys()), weights=probs.values())[0]
            words.append(next_word)
            current_word = next_word
        else:
            break
    return " ".join(words)

In [95]:
generate("never", cfd, 10)

'never stopped till we to honour , and the United'

Составим пары на основе триграмм. Первый элемент в паре – кортеж из первых двух слов триграммы, второй элемент – слово, которое стоит на третьем месте триграммы.

In [ ]:
trigrams = nltk.trigrams(text)
trigrams_pairs = (((w0, w1), w2) for w0, w1, w2 in trigrams)
cfd_trigrams = nltk.ConditionalFreqDist(trigrams_pairs)

In [103]:
def _generate(first_word: str, cfd: nltk.ConditionalFreqDist) -> str:
    probs = cfd[first_word]
    if not probs.keys():
        return
    next_word = random.choices(list(probs.keys()), weights=probs.values())[0]
    return next_word


def improved_generate(
    first_word: str,
    cfd_bigrams: nltk.ConditionalFreqDist,
    cfd_trigrams: nltk.ConditionalFreqDist,
    word_count: int = 10,
) -> str:
    words = [first_word]
    second_word = _generate(first_word, cfd_bigrams)
    if second_word is None:
        return words
    words.append(second_word)
    pair = (first_word, second_word)
    current_word = second_word
    for _ in range(word_count - 1):
        probs = cfd_trigrams[pair]
        if not probs.keys():
            break
        next_word = random.choices(list(probs.keys()), weights=probs.values())[0]
        words.append(next_word)
        pair = (current_word, next_word)
        current_word = next_word

    return " ".join(words)

In [98]:
improved_generate("never", cfd, cfd_trigrams, 10)

'never forget being called to King David in his great captain'

На триграммах текст и правда выглядит осознаннее, хотя до идеала все еще далеко.

### Задание 2

In [49]:
nltk.download('cmudict')
url = 'https://www.gutenberg.org/cache/epub/72839/pg72839.txt'
request = urllib.request.urlopen(url)
text = request.read().decode('utf-8')

[nltk_data] Downloading package cmudict to
[nltk_data]     /Users/ivsidorov/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [50]:
from nltk.corpus import cmudict


phon_dict = cmudict.dict()

Попробуем удовлетворить минимальные требования - чтобы последние две фонемы совпадали, а также количество слогов было одинаковым:

In [84]:
from typing import Dict, List


def _num_syllables(word: str, phon_dict: Dict[str, List]) -> int:
    return [
        len(list(y for y in x if y[-1].isdigit())) for x in phon_dict[word.lower()]
    ][0]


def find_rhymes(word: str, phon_dict: Dict[str, List]) -> List:
    word = word.lower().strip()
    try:
        word_transcription = phon_dict[word][0]
    except KeyError:
        return "Sorry, I don't know that word:("
    if len(word_transcription) > 1:
        word_ending = (word_transcription[-1], word_transcription[-2])
    else:
        word_ending = word_transcription[-1]
    word_num_syll = _num_syllables(word, phon_dict)

    rhymes = []
    for key, transcriptions in phon_dict.items():
        for transcription in transcriptions:
            try:
                if len(transcription) > 1:
                    ending = (transcription[-1], transcription[-2])
                else:
                    ending = transcription[-1]

                if (ending == word_ending) and (
                    _num_syllables(key, phon_dict) == word_num_syll
                ):
                    rhymes.append(key)
            except IndexError:
                continue
    return rhymes

In [98]:
find_rhymes("love", phon_dict=phon_dict)

['dove', 'glove', 'gov', 'love', 'of', 'shove']